In [1]:
import sys
sys.path.append('/Users/anov/IdeaProjects/svoe')

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import data_eng.common as de

In [4]:
%%time

files = de.list_files('s3://svoe.test.1/parquet/BINANCE_FUTURES/l2_book/ADA-USDT/')
files

CPU times: user 7.24 s, sys: 318 ms, total: 7.56 s
Wall time: 41.6 s


['svoe.test.1/parquet/BINANCE_FUTURES/l2_book/ADA-USDT/BINANCE_FUTURES-l2_book-ADA-USDT-1622120422.parquet',
 'svoe.test.1/parquet/BINANCE_FUTURES/l2_book/ADA-USDT/BINANCE_FUTURES-l2_book-ADA-USDT-1622120452.parquet',
 'svoe.test.1/parquet/BINANCE_FUTURES/l2_book/ADA-USDT/BINANCE_FUTURES-l2_book-ADA-USDT-1622120482.parquet',
 'svoe.test.1/parquet/BINANCE_FUTURES/l2_book/ADA-USDT/BINANCE_FUTURES-l2_book-ADA-USDT-1622120512.parquet',
 'svoe.test.1/parquet/BINANCE_FUTURES/l2_book/ADA-USDT/BINANCE_FUTURES-l2_book-ADA-USDT-1622120542.parquet',
 'svoe.test.1/parquet/BINANCE_FUTURES/l2_book/ADA-USDT/BINANCE_FUTURES-l2_book-ADA-USDT-1622120572.parquet',
 'svoe.test.1/parquet/BINANCE_FUTURES/l2_book/ADA-USDT/BINANCE_FUTURES-l2_book-ADA-USDT-1622120602.parquet',
 'svoe.test.1/parquet/BINANCE_FUTURES/l2_book/ADA-USDT/BINANCE_FUTURES-l2_book-ADA-USDT-1622120632.parquet',
 'svoe.test.1/parquet/BINANCE_FUTURES/l2_book/ADA-USDT/BINANCE_FUTURES-l2_book-ADA-USDT-1622120662.parquet',
 'svoe.test.1/parqu

In [5]:
from dask.distributed import Client, LocalCluster
import dask.dataframe as dd
import dask

In [6]:
cluster = LocalCluster()
cluster

/usr/local/lib/python3.9/site-packages/distributed/node.py:160: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 62202 instead
  warnings.warn(


In [7]:
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:62202/status,
Dashboard: http://127.0.0.1:62202/status,Workers: 4
Total threads: 4,Total memory: 8.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:62203,Workers: 4
Dashboard: http://127.0.0.1:62202/status,Total threads: 4
Started: Just now,Total memory: 8.00 GiB
Comm: tcp://192.168.1.70:62215,Total threads: 1
Dashboard: http://192.168.1.70:62219/status,Memory: 2.00 GiB
Nanny: tcp://127.0.0.1:62205,


In [146]:
paths=['s3://' + file for file in files]

In [152]:
folder = 's3://svoe.test.1/parquet/BINANCE_FUTURES/l2_book/ADA-USDT/'

In [166]:
dates = de.group_by_date(paths)

In [170]:
lens = {k: len(v) for k, v in dates.items()}
lens

{'2021-05-27': 9,
 '2021-06-30': 1653,
 '2021-07-01': 2878,
 '2021-07-02': 2880,
 '2021-07-03': 2880,
 '2021-07-04': 2872,
 '2021-07-05': 2872,
 '2021-07-06': 2879,
 '2021-07-07': 2880,
 '2021-07-08': 2880,
 '2021-07-09': 2879,
 '2021-07-10': 2880,
 '2021-07-11': 2878,
 '2021-07-12': 2880,
 '2021-07-13': 939}

In [172]:
d = dask.delayed(dd.read_parquet)(dates['2021-07-01'])
d

Delayed('read_parquet-77f45025-7d1e-4f6c-9059-c39b0c2c4fdb')

In [186]:
%%time
res = d.compute()
res

CPU times: user 21.4 s, sys: 1.91 s, total: 23.3 s
Wall time: 1min 19s


,timestamp,receipt_timestamp,delta,side,price,size
npartitions=2878,,,,,,
,float64,float64,object,object,float64,float64
,...,...,...,...,...,...
...,...,...,...,...,...,...
,...,...,...,...,...,...
,...,...,...,...,...,...


In [185]:
res = res.repartition(npartitions=10)
res

,timestamp,receipt_timestamp,delta,side,price,size
npartitions=10,,,,,,
,float64,float64,object,object,float64,float64
,...,...,...,...,...,...
...,...,...,...,...,...,...
,...,...,...,...,...,...
,...,...,...,...,...,...


In [187]:
cluster.close()



distributed.client - ERROR - Failed to reconnect to scheduler after 30.00 seconds, closing client
_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
asyncio.exceptions.CancelledError
